In [60]:
A_cont, B_cont, C_cont, D_cont = obter_forma_canonica_controlavel(num=[28.08], den=[1, 3, 20.22])


IndexError: index 1 is out of bounds for axis 0 with size 1

# Funções feitas sob medida

In [59]:
import numpy as np
import control as ct


def obter_forma_canonica_controlavel(num, den):

    # Criar o objeto TransferFunction
    tf = ct.TransferFunction(num, den)

    # Converter para espaço de estados
    sys = ct.tf2ss(tf)

    n = sys.A.shape[0]  # Ordem do sistema
    m = sys.B.shape[1]  # Número de entradas

    A_cont = np.zeros((n, n))
    B_cont = np.zeros((n, m))
    C_cont = np.zeros((1, n))
    D_cont = sys.D

    # Coeficientes do denominador (parte com s) do sistema original
    alpha = np.flip(tf.den[0][0], axis=0)[1:]

    # Coeficientes do numerador (parte sem s) do sistema original
    beta = np.flip(tf.num[0][0], axis=0)

    # # Preenchendo as matrizes A_cont, B_cont e C_cont
    # for i in range(n):
    #     A_cont[i, i] = -alpha[i]
    #     if i < n-1:
    #         A_cont[i+1, i] = 1
    #     C_cont[0, i] = beta[i]

    # B_cont[0, :] = beta[-1]

    # Preenchendo as matrizes A_cont, B_cont e C_cont
    for i in range(n):
        A_cont[i, i] = -alpha[i]
        if i < n-1:
            A_cont[i+1, i] = 1
        B_cont[i, :] = beta[i]  # Alteração: preencher a matriz B_cont
        if i < m:
            C_cont[0, i] = beta[i]  # Alteração: preencher a matriz C_cont

    return A_cont, B_cont, C_cont, D_cont




### LIXo

def calcular_propriedades(matriz):
    # Obter os autovalores e autovetores
    autovalores, autovetores = np.linalg.eig(matriz.A)

    # Calcular o posto da matriz
    posto = np.linalg.matrix_rank(matriz.A)

    # print(f'\n autovalores : \n {autovalores}')
    # print(f'\n autovetores : \n {autovetores}')
    # print(f'\n posto : \n')
    # posto

    # Retornar as propriedades calculadas
    return autovalores, autovetores, posto

def calcular_canonicas_autovalores(num, den):
    # Criar a matriz A
    n = len(den) - 1
    A = np.zeros((n, n))
    A[:-1, 1:] = np.eye(n - 1)
    A[-1, :] = -np.array(den[1:])

    # Criar a matriz B
    B = np.zeros((n, 1))
    B[-1] = 1

    # Criar a matriz C
    C = np.zeros((1, n))
    C[0, :] = np.array(num) - np.array(den[1:]) * np.array(num[-1])

    # Criar a matriz D
    D = np.array(num[-1])

    # Calcular a forma canônica controlável
    controlavel_A = np.block([[A, B], [np.zeros((1, n)), 1]])
    controlavel_B = np.zeros((n + 1, 1))
    controlavel_B[-1] = 1

    # Calcular a forma canônica observável
    observavel_A = np.block([[A, np.zeros((n, 1))], [C, 1]])
    observavel_C = np.zeros((1, n + 1))
    observavel_C[0, :-1] = C

    # Calcular os autovalores
    autovalores_controlavel = np.linalg.eigvals(controlavel_A)
    print(autovalores_controlavel)
    autovalores_observavel = np.linalg.eigvals(observavel_A)
    print(autovalores_observavel)

    # Criar o objeto StateSpace
    controlavel_ss = ct.StateSpace(controlavel_A, controlavel_B, np.eye(n), np.zeros((n, 1)))
    print(controlavel_ss)
    observavel_ss = ct.StateSpace(observavel_A, np.zeros((n, 1)), observavel_C, np.zeros((1, 1)))
    print(observavel_ss)

    return controlavel_ss, observavel_ss, autovalores_controlavel, autovalores_observavel





